In [1]:
import pandas as pd
import numpy as np
Df = pd.read_csv('featured-dataset_new_KNN.csv',low_memory=False)
Df['mon_num'] = Df['month'].apply(lambda row: row/12.)
Df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1495814 entries, 0 to 1495813
Data columns (total 22 columns):
Unnamed: 0        1495814 non-null int64
Unnamed: 0.1      1495814 non-null int64
year              1495814 non-null int64
month             1495814 non-null int64
day               1495814 non-null int64
time_cat          1495814 non-null object
time_num          1495814 non-null float64
time_cos          1495814 non-null float64
time_sin          1495814 non-null float64
day_cat           1495814 non-null object
day_num           1495814 non-null float64
day_cos           1495814 non-null float64
day_sin           1495814 non-null float64
weekend           1495814 non-null int64
x_start           1495814 non-null float64
y_start           1495814 non-null float64
z_start           1495814 non-null float64
location_start    1495814 non-null object
location_end      1495814 non-null object
end_lat           1495814 non-null float64
end_lon           1495814 non-null float64


In [2]:
import matplotlib.pyplot as plt
import geohash

In [3]:
Df = Df.drop(Df.columns[0],axis=1)
Df.head(5)
# year mon day time_cat time_num time_cos time_sin day_cat day_num day_cos day_sin weekend x_sta y_sta z_sta lostart loend
# 0    1   2   3         4        5        6        7       8       9       10     11      12     13   14     15      16
# start_lat start_lon end_lat end_lon
#  17        18        19      20 

,Unnamed: 0.1,year,month,day,time_cat,time_num,time_cos,time_sin,day_cat,day_num,...,day_sin,weekend,x_start,y_start,z_start,location_start,location_end,end_lat,end_lon,mon_num
0,0,2018,1,20,10:0,0.425694,-0.892979,0.450098,Saturday,0.775099,...,-0.987591,1,-0.055090,0.712922,0.699076,wqp25w569,wqp25tkvt,33.779811,111.605885,0.083333
1,1,2018,2,12,17:30,0.736111,-0.087156,-0.996195,Monday,0.105159,...,0.613695,0,0.746854,-0.615984,-0.250546,ww4nj3h7m,ww4nj3u7r,34.814875,115.549374,0.166667
2,2,2018,2,13,14:45,0.619444,-0.731354,-0.681998,Tuesday,0.231349,...,0.993142,0,0.762286,-0.641939,-0.082670,ww4jj4hfq,ww4nj9e9j,34.813136,115.559243,0.166667
3,3,2018,2,13,17:15,0.724306,-0.160743,-0.986996,Tuesday,0.246329,...,0.999734,0,0.740919,-0.620137,-0.257816,ww4nj4rph,ww4nj9e9h,34.813141,115.559217,0.166667
4,4,2018,2,13,18:0,0.754167,0.026177,-0.999657,Tuesday,0.250595,...,0.999993,0,0.752481,-0.608090,-0.252980,ww4nj9edj,ww4muu95u,34.786126,115.874361,0.166667


In [4]:
Df.shape

(1495814, 21)

In [5]:
columns_all_features = Df.columns
columns_X = ['day_num', 'mon_num', 'x_start', 'y_start', 'z_start']
columns_y = ['end_lat', 'end_lon']
testDf_1 = Df.sample(frac=0.5)
testDf_2 = Df.sample(frac=0.5)
testDf_3 = Df.sample(frac=0.5)
X_1 = testDf_1[columns_X]
y_1 = testDf_1[columns_y]
X_2 = testDf_2[columns_X]
y_2 = testDf_2[columns_y]
X_3 = testDf_3[columns_X]
y_3 = testDf_3[columns_y]

In [11]:
from sklearn.model_selection import train_test_split
X1_train, X1_test, y1_train, y1_test = train_test_split(X_1, y_1, test_size=0.2)
X2_train, X2_test, y2_train, y2_test = train_test_split(X_2, y_2, test_size=0.2)
X3_train, X3_test, y3_train, y3_test = train_test_split(X_3, y_3, test_size=0.2)

In [12]:
print ('X1: ({}, {})'.format(*X_1.shape))
print ('y1: ({}, {})'.format(*y_1.shape))
print ('X1_train: ({}, {})'.format(*X1_train.shape))
print ('y1_train: ({}, {})'.format(*y1_train.shape))
print ('X1_test: ({}, {})'.format(*X1_test.shape))
print ('y1_test: ({}, {})'.format(*y1_test.shape))
print ('X2: ({}, {})'.format(*X_2.shape))
print ('y2: ({}, {})'.format(*y_2.shape))
print ('X2_train: ({}, {})'.format(*X2_train.shape))
print ('y2_train: ({}, {})'.format(*y2_train.shape))
print ('X2_test: ({}, {})'.format(*X2_test.shape))
print ('y2_test: ({}, {})'.format(*y2_test.shape))

X1: (747907, 5)
y1: (747907, 2)
X1_train: (598325, 5)
y1_train: (598325, 2)
X1_test: (149582, 5)
y1_test: (149582, 2)
X2: (747907, 5)
y2: (747907, 2)
X2_train: (598325, 5)
y2_train: (598325, 2)
X2_test: (149582, 5)
y2_test: (149582, 2)


In [13]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [14]:
from sklearn.model_selection import GridSearchCV
# Set the parameters by cross-validation
tuned_parameters = {'n_estimators': [25], 'max_depth': [None], 'min_samples_split': [5]}


# clf = ensemble.RandomForestRegressor(n_estimators=500, n_jobs=1, verbose=1)
gridCV = GridSearchCV(RandomForestRegressor(), tuned_parameters, cv=5, n_jobs=-1, verbose=1)
gridCV.fit(X1_train, y1_train)
print (gridCV.best_estimator_)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:  4.5min remaining:  6.7min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  5.3min finished


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=5,
           min_weight_fraction_leaf=0.0, n_estimators=25, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)


In [15]:
reg = gridCV.best_estimator_
training_accuracy = reg.score(X1_train, y1_train)
valid_accuracy = reg.score(X1_test, y1_test)
rmsetrain = np.sqrt(mean_squared_error(reg.predict(X1_train),y1_train))
rmsevalid = np.sqrt(mean_squared_error(reg.predict(X1_test),y1_test))
print (" R^2 (train) = %0.6f, R^2 (valid) = %0.6f, RMSE (train) = %0.6f, RMSE (valid) = %0.6f" % (training_accuracy, valid_accuracy, rmsetrain, rmsevalid))

 R^2 (train) = 0.973379, R^2 (valid) = 0.889203, RMSE (train) = 1.194192, RMSE (valid) = 2.431386


In [16]:
from sklearn.externals import joblib
joblib.dump(reg, 'conbinRF01_1106.pkl') 

['conbinRF01_1106.pkl']

In [20]:
gridCV.fit(X2_train, y2_train)
print (gridCV.best_estimator_)
reg = gridCV.best_estimator_
training_accuracy = reg.score(X2_train, y2_train)
valid_accuracy = reg.score(X2_test, y2_test)
rmsetrain = np.sqrt(mean_squared_error(reg.predict(X2_train),y2_train))
rmsevalid = np.sqrt(mean_squared_error(reg.predict(X2_test),y2_test))
print (" R^2 (train) = %0.6f, R^2 (valid) = %0.6f, RMSE (train) = %0.6f, RMSE (valid) = %0.6f" % (training_accuracy, valid_accuracy, rmsetrain, rmsevalid))
joblib.dump(reg, 'conbinRF02_1106.pkl') 

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:  9.9min remaining: 14.8min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed: 10.4min finished


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=5,
           min_weight_fraction_leaf=0.0, n_estimators=25, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)
 R^2 (train) = 0.973326, R^2 (valid) = 0.892392, RMSE (train) = 1.195899, RMSE (valid) = 2.396623


['conbinRF02_1106.pkl']

In [21]:
gridCV.fit(X3_train, y3_train)
print (gridCV.best_estimator_)
reg = gridCV.best_estimator_
training_accuracy = reg.score(X3_train, y3_train)
valid_accuracy = reg.score(X3_test, y3_test)
rmsetrain = np.sqrt(mean_squared_error(reg.predict(X3_train),y3_train))
rmsevalid = np.sqrt(mean_squared_error(reg.predict(X3_test),y3_test))
print (" R^2 (train) = %0.6f, R^2 (valid) = %0.6f, RMSE (train) = %0.6f, RMSE (valid) = %0.6f" % (training_accuracy, valid_accuracy, rmsetrain, rmsevalid))
joblib.dump(reg, 'conbinRF03_1106.pkl') 

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed: 10.3min remaining: 15.5min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed: 11.3min finished


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=5,
           min_weight_fraction_leaf=0.0, n_estimators=25, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)
 R^2 (train) = 0.973886, R^2 (valid) = 0.890960, RMSE (train) = 1.182947, RMSE (valid) = 2.418690


['conbinRF03_1106.pkl']

In [ ]:
importances = reg.feature_importances_
std = np.std([tree.feature_importances_ for tree in reg.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(X.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

# Plot the feature importances of the forest
feature_names = X_test.columns
plt.figure()
plt.title("Feature importances")
plt.bar(range(X_test.shape[1]), importances[indices],
       color="r", yerr=std[indices], align="center")
plt.xticks(range(X_test.shape[1]), feature_names)
plt.xlim([-1, X_test.shape[1]])
plt.show()

In [ ]:
sampleds = pd.DataFrame(Df, columns=(columns_X + columns_y))
sampleds = sampleds.sample(10)
sampleds

In [ ]:
y_pred = reg.predict(sampleds.iloc[:,:-2])
y_pred

In [ ]:
from sklearn.externals import joblib
joblib.dump(reg, 'random_forest_model_1151724.pkl') 